In [221]:
import pandas as pd
import numpy as np

In [347]:
folder = '.\\input_files\\'

file = 'documents_standard-charges_41-0695602_Essentia-Health-St.-Josephs-Medical-Center_standardcharges.csv'

In [ ]:
mapping = {
    'CDM Price': 'gross',
    'Discounted Self Pay/Cash Price': 'cash',
    'Inpatient De-Identified Minimum Negoticated Rate': 'min',
    'Inpatient De-Identified Maximum Negoticated Rate': 'max',
    'Outpatient De-Identified Minimum Negoticated Rate': 'min',
    'Outpatient De-Identified Maximum Negoticated Rate': 'max',
    'Median Charges': 'gross',
    
}


In [348]:
df = pd.read_csv(folder + file, dtype=str, encoding='ansi', skiprows=3)

# Splitting the df

In [349]:
# Initialize an empty list to hold the separate DataFrames
df_list = []

# Initialize a counter to keep track of the start of each DataFrame
start_row = 0

# Loop through the rows in the DataFrame
for i, row in df.iterrows():

    # Check if the row is completely null
    if row.isnull().all():
        # If so, create a new DataFrame from the previous null row to this one
        new_df = df.iloc[start_row:i]
        
        if len(df_list) > 0 and len(new_df) > 0:
            new_df.columns = new_df.iloc[0]
            new_df = new_df.iloc[1:]
        
        if len(new_df) > 0:
            # Add the new DataFrame to the list
            df_list.append(new_df)

        # Update the start row to the current row + 1
        start_row = i + 1

dfs = []
# Assign each DataFrame in the list to a separate variable
for i, df in enumerate(df_list):
    globals()[f"df_{i+1}"] = df
    dfs.append(df)

num_df = len(df_list)

- `df_2` is the Per Diem Rate df
- `df_3` is DRG
- `df_4` is APRDRG


# df_1 Stuff

In [239]:
df_1.rename(columns={
    'EAP Code (Internal Use)': 'local_code',
    'CPT/HCPCS Code': 'code',
    'Procedure Description (Charge)': 'description',
    'Raw 11 Digit NDC or Primary Ext Id': 'ndc',
    'Raw 11 Digit NDC or Primary Ext Id ': 'ndc',
    'Revenue Code': 'rev_code'
}, inplace=True)

In [240]:
cols = df_1.columns.tolist()
print(cols)
id_vars = cols[:5]
value_vars = cols[5:]

df_1 = pd.melt(df_1, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

['8635790', '86357', '(NK) CELLS, TOTAL COUNT - REF', 'Unnamed: 3', '300', '$195.00 ', '$146.25 ', '$0.00 ', '$0.00 .1', '$58.31 ', '$185.25 ', 'N/A', '$156.00 ', 'N/A.1', 'N/A.2', 'N/A.3', 'N/A.4', 'N/A.5', 'N/A.6', 'N/A.7', 'N/A.8', 'N/A.9', 'N/A.10', 'N/A.11', 'N/A.12', 'N/A.13', '$113.19 ', 'N/A.14', 'N/A.15', 'N/A.16', 'N/A.17', 'N/A.18', 'N/A.19', 'N/A.20', '$153.32 ', 'N/A.21', 'N/A.22', 'N/A.23', '$165.75 ', 'N/A.24', 'N/A.25', 'N/A.26', 'N/A.27', 'N/A.28', '$167.70 ', 'N/A.29', '$87.17 ', 'N/A.30', '$87.17 .1', 'N/A.31', 'N/A.32', 'N/A.33', '$132.60 ', 'N/A.34', 'N/A.35', 'N/A.36', '$146.25 .1', 'N/A.37', 'N/A.38', 'N/A.39', '$185.25 .1', 'N/A.40', 'N/A.41', 'N/A.42', 'N/A.43', 'N/A.44', 'N/A.45', 'N/A.46', '$136.50 ', 'N/A.47', '$58.31 .1', 'N/A.48', 'N/A.49', 'N/A.50', '$163.61 ', 'N/A.51', '$73.32 ', 'N/A.52', '$146.64 ', 'N/A.53', 'N/A.54', 'N/A.55', '$165.75 .1', 'N/A.56', 'N/A.57', 'N/A.58', '$178.52 ', 'N/A.59', '$172.03 ', 'N/A.60', 'N/A.61', 'N/A.62', 'N/A.63', 'N/A.6

In [227]:
df_1['payer_category'] = df_1['payer_name'].map(mapping).fillna('payer')

In [228]:
df_1.loc[df_1['payer_name'].str.lower().str.contains('inpatient'), 'setting'] = 'inpatient'

In [229]:
df_1['setting'] = np.where(
    df_1['payer_name'].str.lower().str.contains('inpatient'), 'inpatient',
    np.where(
        df_1['payer_name'].str.lower().str.contains('outpatient'), 'outpatient', pd.NA)
)

In [230]:
df_1['rev_code'] = df_1['rev_code'].str.zfill(4)

KeyError: 'rev_code'

In [ ]:
df_1.dropna(subset='standard_charge', inplace=True)

In [ ]:
df_1.loc[(~df_1['code'].isna()) & (df_1['code'].str.match(r'^(?:[A-Z][0-9]{4}|[0-9]{5}|[0-9]{4}[A-Z])$')), 'hcpcs_cpt'] = df_1['code']

# Classify each DF

In [375]:
apr_df = False
drg_df = False
op_df = False
op_df1 = False
per_diem_df = False


for i, df in enumerate(dfs):
    if not i:
        print(df.columns.tolist())
    if i:
        # Have to remove nan columns
        df.columns = [str(col) if not pd.isna(col) else '' for col in df.columns]
        print(df.columns.tolist())
        if '' in df.columns:
            df = df.drop([''], axis=1)
        first_col = df.columns[0]
        if first_col == 'DRG Type':
            drg_df = df
        elif first_col == 'APR -DRG':
            apr_df = df
        elif first_col == 'Per Diem Rate':
            per_diem_df = df
        elif first_col == 'OP Type':
            if type(op_df) == bool:
                op_df = df
            else:
                op_df1 = df

['EAP Code (Internal Use)', 'CPT/HCPCS Code', 'Procedure Description (Charge)', 'Raw 11 Digit NDC or Primary Ext Id ', 'Revenue Code', 'CDM Price', 'Discounted Self Pay/Cash Price', 'Inpatient De-Identified Minimum Negoticated Rate', 'Inpatient De-Identified Maximum Negoticated Rate', 'Outpatient De-Identified Minimum Negoticated Rate', 'Outpatient De-Identified Maximum Negoticated Rate', "AMERICA'S PPO-Inpatient", "AMERICA'S PPO-Outpatient", 'BCBS OF MN Commercial-Inpatient', 'BCBS OF MN Commercial-Outpatient', 'BCBS MN ST LOUIS CO EMP-Inpatient', 'BCBS MN ST LOUIS CO EMP-Outpatient', 'BCBS MN MEDICARE ADVANTAGE -Inpatient', 'BCBS MN MEDICARE ADVANTAGE -Outpatient', 'BCBS MN SECURE BLUE MSHO-Inpatient', 'BCBS MN SECURE BLUE MSHO-Outpatient', 'BLUE PLUS PMAP PCC PRIME-Inpatient', 'BLUE PLUS PMAP PCC PRIME-Outpatient', 'SANFORD HEALTH BCBS of ND MA EXPANSION-Inpatient', 'SANFORD HEALTH BCBS of ND MA EXPANSION-Outpatient', 'BCBS OF ND MEDICARE ADVANTAGE-Inpatient', 'BCBS OF ND MEDICARE A

# Per diem df Stuff

In [340]:
# Have to use type here because dataframes don't have a truth value
if type(per_diem_df) != bool:
    # per_diem_df['contracting_method'] = 'per diem'
    # per_diem_df['description'] = 'per diem rate'

    # cols = per_diem_df.columns.tolist()
    # id_vars = ['description', 'contracting_method']
    # value_vars = cols[1:-2]
    # per_diem_df = pd.melt(per_diem_df, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')
    
    per_diem_df.dropna(subset='standard_charge', inplace=True)

    per_diem_df['payer_category'] = per_diem_df['payer_name'].map(mapping).fillna('payer')

# apr-drg df

In [376]:
# Have to use type here because dataframes don't have a truth value
if type(apr_df) != bool:
    apr_df.rename(columns={
        'APR -DRG': 'line_type',
        'APRDRG & Severity': 'apr_drg',
        'APR Description': 'description',
    }, inplace=True)

    apr_df.loc[apr_df['apr_drg'].str.len() == 2, 'apr_drg'] = apr_df['apr_drg'].str.zfill(4)
    apr_df['apr_drg'] = apr_df['apr_drg'].str[:3] + '-' + apr_df['apr_drg'].str[-1]

    cols = apr_df.columns.tolist()
    id_vars = cols[:3]
    value_vars = cols[3:]
    
    apr_df = pd.melt(apr_df, id_vars=id_vars, value_vars=value_vars, var_name='payer_name', value_name='standard_charge')

In [377]:
apr_df.dropna(subset='standard_charge')

,line_type,apr_drg,description,payer_name,standard_charge
15,APR,005-4,TRACHEOSTOMY W MV 96+ HOURS W/O EXTENSIVE PROC...,Median Charges,"$532,493.50"
89,APR,452-2,CVA & PRECEREBRAL OCCLUSION W INFARCT,Median Charges,"$18,524.00"
90,APR,453-3,CVA & PRECEREBRAL OCCLUSION W INFARCT,Median Charges,"$29,970.25"
112,APR,511-1,VIRAL MENINGITIS,Median Charges,"$16,311.50"
118,APR,523-3,ALTERATION IN CONSCIOUSNESS,Median Charges,"$22,710.50"
...,...,...,...,...,...
138595,APR,951-4,MODERATELY EXTENSIVE PROCEDURE UNRELATED TO PR...,UCARE Medicaid-Inpatient,"$45,339.71"
138596,APR,952-1,NONEXTENSIVE PROCEDURE UNRELATED TO PRINCIPAL ...,UCARE Medicaid-Inpatient,"$9,812.31"
138597,APR,952-2,NONEXTENSIVE PROCEDURE UNRELATED TO PRINCIPAL ...,UCARE Medicaid-Inpatient,"$13,813.28"
138598,APR,952-3,NONEXTENSIVE PROCEDURE UNRELATED TO PRINCIPAL ...,UCARE Medicaid-Inpatient,"$22,384.98"
